![banner](../logo/banner.png)

# 3.8 Sensores analogicos

Vamos a simular sensores analogicos usando un potenciomentro, el cual su variacion hara que realicemos una accion determinada.

## 3.8.1 Sensor de distancia infrarrojo

Un sensor infrarrojo la señal que nos entrega es la relacion de que tanta luz incide en el fototransistor, la cual es emitida por un led infrarrojo. Es decir, si la luz que emite el led llega al fototransistor debemos realizar o no una accion.

![irda](imgs/infrarrojo_analogico.jpg)

Ejemplo de un circuito de sensor infrarro simple:

![circuito irda](imgs/irda_circuito.png)

### 3.8.1 Aplicacion de sensores infrarrojos

**Ejemplo 1. **

## 3.8.2 Sensor de corriente 

Tenemos vario sensores de corrientes, aqui muestro el sensor de efecto Hall y una bobina.
Como tal el sensor hall detecta campos magneticos, no corriente, pero como un efecto del paso de la corriente genera un campo magnetico por ello se utiliza. 
Al pasar una corriente se genera un campo magnetico y si se le acerca una bobina se genera el efecto de induccion, lo que causa que en dicha bobina se genere una corriente equivalente a la corriente que pasa en el primer conductor.

En ambos casos podemos determinar la cantidad de corriente que esta pasando en dicho conductor gracias a su campo magnetico.

![hall](imgs/hall_analogico.jpeg)

![bobina](imgs/corriente_sensor.jpg)

### 3.8.2.1 Aplicacion de sensores de corriente

**Ejemplo X. **



---

## Practicas - sensores Analogicos

1. Simular un sensor de distancia ultrasonico, que cuando identifique que el objeto esta a una distancia menor a 10cm haga parpadear una barra de leds.
2. Simular un sensor de luz junto con un sensor de presencia (digital). Dicho sensores trabajan a la par, si hay poca luz y existe presencia en el cuarto se debe encender un foco. En caso que no exista presencia sin importar la intensidad de la luz, el foco no enciende.
3. Simular un sensor infrarrojo para accionar el movimiento de un servomotor, entre mas luz incida debe incrementar el angulo del servomotor. Es decir, entre mas cerca se incrementa el angulo, entre mas lejos es menor el angulo.

Realizado por Docente: [Alejandro Leyva](https://www.alejandro-leyva.com/)

[Mecatrónica 85](https://mecatronica85.com/)

[fb/mecatronica85](https://www.facebook.com/mecatronica85)